In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd
import numpy as np
import joblib
import math
import decimal

Criar argumentos

In [ ]:
if not 'param_league' in locals():
    param_league = 'Liga_Portugal'
if not 'param_league_short' in locals():
    param_league_short = 'lp'
if not 'param_season' in locals():
    param_season = '_2023_24'
if not 'param_final_flag' in locals():
    param_final_flag = False
if not 'param_odd_bet_limit' in locals():
    param_odd_bet_limit = 1.2
if not 'param_number_matches_week' in locals():
    param_number_matches_week = 9
if not 'param_itera_jornada' in locals():
    param_itera_jornada = 2
if not 'param_list_derby_teams' in locals():
    param_list_derby_teams = []
if not 'param_predicts_path' in locals():
    param_predicts_path = ''
if not 'param_uniform_bet_size' in locals():
    param_uniform_bet_size = False
if not 'param_print_slip' in locals():
    param_print_slip = False


Lê os parametros e escreve nas respectivas variáveis

In [ ]:
liga =  param_league
liga_short = param_league_short
epoca = param_season
final_flag = param_final_flag
odd_bet_limit = param_odd_bet_limit
number_matches_week = param_number_matches_week
jornada = param_itera_jornada
list_derby_teams = param_list_derby_teams
uniform_bet_size = param_uniform_bet_size
print_slip = param_print_slip

Definição de Constantes

In [ ]:
MAIN_PATH = '/content/drive/MyDrive/'
MAIN_PATH_LEAGUE  = (MAIN_PATH + 'Data_Football/{0}/').format(liga)
TOTAL_FUNDS_DEFAULT = 50.0

Invocar notebooks de funções e de constantes

In [ ]:
functions_path = MAIN_PATH + 'Colab Notebooks/Bet_Functions.ipynb'

%run '{functions_path}'


Definição dos paths em função dos parametros

In [ ]:
if param_predicts_path == '' :
  summary_path = MAIN_PATH_LEAGUE + 'Predicts/Summary_{0}{1}.csv'.format(liga_short,epoca)
  slip_path_generico = MAIN_PATH_LEAGUE + 'Predicts/GW_{0}'
else:
  summary_path = param_predicts_path + '/Summary_{0}{1}.csv'.format(liga_short,epoca)
  slip_path_generico = param_predicts_path + '/GW_{0}'

raw_data_path = MAIN_PATH_LEAGUE + 'Raw/{0}{1}.csv'.format(liga,epoca)
clean_data_path = MAIN_PATH_LEAGUE + 'Clean/{0}{1}_clean.csv'.format(liga,epoca)
model_path = MAIN_PATH_LEAGUE+ 'Model/{0}_model.joblib'.format(liga_short)

Get proxima jornada e total funds

In [ ]:

total_funds_previousweek = TOTAL_FUNDS_DEFAULT

print('in: ' + summary_path)

# Se ficheiro summary existe calcular jornada actual e fundos actuais
if os.path.exists(summary_path) == True:
  dfSummary=pd.read_csv(summary_path)
  dfLastWeek = dfSummary.iloc[-1:]
  jornada = int(dfSummary.iloc[-1:]['game_week'] + 1)
  total_funds_previousweek = float(dfSummary.iloc[-1:]['total_funds_final'])

bet_size = calculate_bet_size(uniform_bet_size, 0.05, total_funds_previousweek)

print( 'Jornada: ' + str(jornada))
print ('total_funds_previousweek = ' + str(total_funds_previousweek))
print ('bet_size = ' + str(bet_size))



NameError: name 'TOTAL_FUNDS_DEFAULT' is not defined

Extração do ficheiro clean

In [ ]:

df=pd.read_csv(raw_data_path)
df_clean=df[[ "GW","HomeTeam",	"AwayTeam",	"FTHG",	"FTAG",	"FTR",	"HTHG",	"HTAG",	"HTR",	"HS",	"AS",	"HST",	"AST",	"HF",	"AF",	"HC",	"AC",'BWH','BWD','BWA']].copy()


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,P1,11/8/2023,20:15,Sp Braga,Famalicao,1.0,2.0,A,1.0,0.0,...,2.24,-1.25,2.08,1.73,2.18,1.75,2.23,1.90,2.07,1.78
1,P1,12/8/2023,15:30,Gil Vicente,Portimonense,5.0,0.0,H,2.0,0.0,...,1.90,-0.50,1.87,2.03,1.86,2.05,1.88,2.06,1.83,2.01
2,P1,12/8/2023,18:00,Farense,Casa Pia,0.0,3.0,A,0.0,1.0,...,1.55,-0.25,2.17,1.75,2.19,1.75,2.20,1.83,2.07,1.77
3,P1,12/8/2023,20:30,Sp Lisbon,Vizela,3.0,2.0,H,2.0,0.0,...,2.37,-1.75,1.87,2.03,1.85,2.05,1.88,2.12,1.81,2.04
4,P1,13/08/2023,15:30,Rio Ave,Chaves,2.0,0.0,H,0.0,0.0,...,1.70,-0.50,1.92,1.98,1.93,1.97,1.95,2.01,1.90,1.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,NaN,NaN,NaN,Arouca,Benfica,NaN,NaN,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,NaN,NaN,NaN,Sp Braga,Guimaraes,NaN,NaN,D,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,NaN,NaN,NaN,Rio Ave,Portimonense,NaN,NaN,H,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,NaN,NaN,NaN,Famalicao,Chaves,NaN,NaN,D,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Limpar nomes dos clubes -Upper Case e limpar espaços

In [ ]:
df_clean.loc[:, ['HomeTeam']] = df_clean["HomeTeam"].apply(str.upper).apply(lambda x: x.replace(' ', ''))
df_clean.loc[:, ['AwayTeam']] = df_clean["AwayTeam"].apply(str.upper).apply(lambda x: x.replace(' ', ''))

Copiar ficheiro final para a directoria clean

In [ ]:
df_clean.to_csv(clean_data_path)


,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,BWH,BWD,BWA
0,SPBRAGA,FAMALICAO,1.0,2.0,A,1.0,0.0,H,13.0,12.0,2.0,4.0,17.0,12.0,3.0,4.0,1.44,4.50,6.50
1,GILVICENTE,PORTIMONENSE,5.0,0.0,H,2.0,0.0,H,8.0,9.0,5.0,3.0,11.0,14.0,1.0,1.0,2.00,3.30,3.60
2,FARENSE,CASAPIA,0.0,3.0,A,0.0,1.0,A,12.0,14.0,5.0,5.0,13.0,13.0,5.0,6.0,2.45,3.00,2.95
3,SPLISBON,VIZELA,3.0,2.0,H,2.0,0.0,H,19.0,10.0,6.0,4.0,16.0,9.0,8.0,3.0,1.18,6.75,16.00
4,RIOAVE,CHAVES,2.0,0.0,H,0.0,0.0,D,21.0,6.0,8.0,1.0,12.0,20.0,14.0,3.0,2.10,3.25,3.50


In [ ]:
dl_lp_fix, dl_lp_match_raw= split_data_fixture_number(jornada, df_clean)

dl_lp_match = dl_lp_match_raw.copy()
dl_lp_match.loc[:, ['HomeTeam']] = dl_lp_match["HomeTeam"].apply(str.upper).apply(lambda x: x.replace(' ', ''))
dl_lp_match.loc[:, ['AwayTeam']] = dl_lp_match["AwayTeam"].apply(str.upper).apply(lambda x: x.replace(' ', ''))
dl_lp_match

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,BWH,BWD,BWA
135,SPLISBON,ESTORIL,NaN,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.20,6.25,10.50
136,BOAVISTA,PORTO,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.50,4.40,1.39
137,FARENSE,GILVICENTE,NaN,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.10,3.60,3.20
138,ESTRELA,VIZELA,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.50,3.30,2.70
139,AROUCA,BENFICA,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.75,4.33,1.39
140,SPBRAGA,GUIMARAES,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.47,4.33,5.50
141,RIOAVE,PORTIMONENSE,NaN,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.85,3.50,4.00
142,FAMALICAO,CHAVES,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.83,3.60,4.00
143,MOREIRENSE,CASAPIA,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.90,3.40,4.00


In [ ]:
all_teams_home = dl_lp_fix.HomeTeam.unique()
all_teams_away = dl_lp_fix.AwayTeam.unique()


all_teams_raw = np.concatenate([all_teams_home, all_teams_away])

all_teams = np.unique(all_teams_raw)

all_teams_stats = []

for team in all_teams:
  team_stats = calc_feats(dl_lp_fix,team)
  all_teams_stats.append(team_stats)


df_all_stats = pd.DataFrame(all_teams_stats)

,team,shots,shots_ot,shots_vs,shots_ot_vs,points,form
0,AROUCA,193.0,70.0,147.0,64.0,16,10
1,BENFICA,254.0,100.0,147.0,47.0,36,11
2,BOAVISTA,142.0,54.0,188.0,80.0,16,1
3,CASAPIA,147.0,50.0,199.0,79.0,16,6
4,CHAVES,187.0,70.0,185.0,86.0,10,3
5,ESTORIL,178.0,77.0,209.0,76.0,17,10
6,ESTRELA,136.0,41.0,211.0,70.0,16,5
7,FAMALICAO,167.0,50.0,157.0,64.0,18,3
8,FARENSE,170.0,61.0,267.0,88.0,18,5
9,GILVICENTE,150.0,66.0,195.0,74.0,15,5


Apply Model

In [ ]:
features = df_all_stats[['shots','shots_ot','shots_vs','shots_ot_vs']]
model = joblib.load(model_path)

df_all_stats['exp_points'] = model.predict(features)

df_all_stats.sort_values('exp_points', ascending=[False])



,team,shots,shots_ot,shots_vs,shots_ot_vs,points,form,exp_points
1,BENFICA,254.0,100.0,147.0,47.0,36,11,36.408847
15,SPBRAGA,247.0,95.0,156.0,62.0,32,12,30.269497
13,PORTO,202.0,79.0,130.0,37.0,34,12,29.841188
16,SPLISBON,206.0,86.0,118.0,44.0,37,9,28.108923
10,GUIMARAES,180.0,60.0,148.0,55.0,29,10,19.522667
0,AROUCA,193.0,70.0,147.0,64.0,16,10,19.254551
11,MOREIRENSE,168.0,53.0,167.0,54.0,26,9,18.701433
5,ESTORIL,178.0,77.0,209.0,76.0,17,10,18.299108
17,VIZELA,164.0,62.0,189.0,63.0,12,3,17.796837
8,FARENSE,170.0,61.0,267.0,88.0,18,5,14.628617


Value Betting
Prever resultados da jornada

In [ ]:
dl_lp_match[['HomeTeam', 'AwayTeam','BWH','BWD','BWA','FTR']]
dl_lp_match

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,BWH,BWD,BWA
135,SPLISBON,ESTORIL,NaN,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.20,6.25,10.50
136,BOAVISTA,PORTO,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.50,4.40,1.39
137,FARENSE,GILVICENTE,NaN,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.10,3.60,3.20
138,ESTRELA,VIZELA,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.50,3.30,2.70
139,AROUCA,BENFICA,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.75,4.33,1.39
140,SPBRAGA,GUIMARAES,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.47,4.33,5.50
141,RIOAVE,PORTIMONENSE,NaN,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.85,3.50,4.00
142,FAMALICAO,CHAVES,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.83,3.60,4.00
143,MOREIRENSE,CASAPIA,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.90,3.40,4.00


In [ ]:
predictions = []

for ind,row in dl_lp_match.iterrows():

  home_team = row['HomeTeam']
  away_team = row['AwayTeam']

  home_exp_points = df_all_stats[
      df_all_stats['team']==home_team]['exp_points'].values[0]
  away_exp_points = df_all_stats[
      df_all_stats['team']==away_team]['exp_points'].values[0]

  derby =  (home_team in list_derby_teams ) & (away_team  in list_derby_teams)

  exp_points_diff_teams = (home_exp_points - away_exp_points)

  if exp_points_diff_teams > 4:
    result = 'H'
    odds = row['BWH']
  elif abs(exp_points_diff_teams) < 4:
     result = 'D'
     odds = row['BWD']
  else:
    result = 'A'
    odds = row['BWA']

  pred = {"HomeTeam" : home_team, "AwayTeam": away_team, "Prediction": result, "Odds":odds, "Derby": derby}

  predictions.append(pred)

NameError: name 'dl_lp_match' is not defined

Build slip

In [ ]:
decimal.getcontext().rounding = decimal.ROUND_HALF_UP


df_slip = pd.DataFrame(predictions)

df_slip = dl_lp_match.merge(df_slip, on = ['HomeTeam', 'AwayTeam'])[['HomeTeam','AwayTeam','Derby','Prediction','Odds','FTR']]

df_slip['BetSize'] = np.where(
     ((df_slip['Odds'] < odd_bet_limit) | (df_slip['Derby']==True))
     ,0
    ,bet_size
)

df_slip['Win'] = np.where(
     (df_slip['Prediction']==df_slip['FTR']) & (df_slip['BetSize'] > 0)
     ,((df_slip['Odds']-1)*bet_size)
     ,np.where(df_slip['BetSize']==0
               ,0
               ,-1*bet_size)
     )

if (print_slip == True):
  print(df_slip)

,HomeTeam,AwayTeam,Prediction,Odds,FTR,BetSize,Win
0,SPLISBON,ESTORIL,H,1.20,H,3.0,0.60
1,BOAVISTA,PORTO,A,1.39,D,3.0,-3.00
2,FARENSE,GILVICENTE,D,3.60,H,3.0,-3.00
3,ESTRELA,VIZELA,A,2.70,D,3.0,-3.00
4,AROUCA,BENFICA,A,1.39,A,3.0,1.17
5,SPBRAGA,GUIMARAES,H,1.47,D,3.0,-3.00
6,RIOAVE,PORTIMONENSE,H,1.85,H,3.0,2.55
7,FAMALICAO,CHAVES,D,3.60,D,3.0,7.80
8,MOREIRENSE,CASAPIA,H,1.90,N,3.0,-3.00


In [ ]:
# gera ficheiro spli raw com os valores da jornada
total_win_provisory = df_slip['Win'].sum()
total_funds_provisory = total_funds_previousweek + total_win_provisory


slip_path = slip_path_generico.format(str(jornada).rjust(2, "0"))

print('total win provisory = ' + str(round(total_win_provisory,2)))
print('total fundos provisory = ' + str(round(total_funds_provisory,2)))

if final_flag == False:
  df_slip.to_csv(slip_path + '_raw.csv')
  print('Comparar valores reais com os do ficheiro ' + slip_path + '_raw.csv' + " e construir ficheiro slip final: " + slip_path + '.csv')
else:
  df_slip.to_csv(slip_path + '.csv')

total win provisory = -2.88
total fundos provisory = 63.31
Comparar valores reais com os do ficheiro /content/drive/MyDrive/Data_Football/Liga_Portugal/Predicts/slip_jornada_16_raw.csv e construir ficheiro slip final: /content/drive/MyDrive/Data_Football/Liga_Portugal/Predicts/slip_jornada_16.csv


Calc sum and append to sumary

In [ ]:

# Se ficheiro slip final não existe chamar a função exit do notebook de funções
if os.path.exists(slip_path + '.csv')==False:
  print('Jornada ' + str(jornada) + ' ainda não foi completa.')
  exit()

#lê o slip desta jornada
df_slip=pd.read_csv(slip_path + '.csv')

#retira o array BetSizes e calcula o numero de apostas efectuadas (todas as apostas com BetSize >0)
arrayBetSize = df_slip['BetSize']
number_bets = arrayBetSize.where(arrayBetSize > 0).count()

#retira o array Win e calcula o total de ganhos (suma) e numero de apostas ganhas (valor >0)
arrayWin = round(df_slip['Win'],2)
gameweek_winning = round(arrayWin.sum(),2)
number_winning_bets = arrayWin.where(arrayWin > 0).count()

#Total dos fundos alocados = Soma vo valor de todas apostas efectuadas
total_funds_allocated = df_slip['BetSize'].sum()

#Fundos finais = fundos totais da semana anterior + ganhos desta semana
total_funds_final = total_funds_previousweek + gameweek_winning

# Retorno sobre o montante apostado
perc_return_on_bet = (0 if total_funds_allocated == 0 else round(100*(gameweek_winning/total_funds_allocated), 2))

# Percentagem de apostas ganhas (numero de apostas ganhas / numero de apostas efectuadaa)
perc_winning_bets = (0 if number_bets == 0 else round(100*(number_winning_bets/number_bets), 2))

# Retorno total  = Fundos finais nesta semana- fundos iniciais / fundos iniciais)
perc_total_return = round(100*(total_funds_final-TOTAL_FUNDS_DEFAULT)/TOTAL_FUNDS_DEFAULT,2)

#contrução do registo a adicionar ao dataframe
game_week_summary = [{'game_week': jornada,
                      'game_week_winning': gameweek_winning,
                      'bet_size': bet_size,
                      'total_funds_allocated': total_funds_allocated,
                      'total_funds': round(total_funds_previousweek,2),
                      'total_funds_final': round(total_funds_final,2),
                      'percent_return_on_bet' : perc_return_on_bet,
                      'percent_total_return': perc_total_return,
                      'number_bets' : number_bets,
                      'number_win_bets': number_winning_bets,
                      'percent_winning_bets': perc_winning_bets}]

df_game_week_summary = pd.DataFrame(game_week_summary)

if os.path.exists(summary_path) == False:
  # criar ficheiro e adidionar registo da jornada ao csv de Sumário com header
  df_game_week_summary.to_csv(summary_path, mode='x', header=True, index=False)
else:
  # adidionar registo da jornada ao csv de Sumário
  df_game_week_summary.to_csv(summary_path, mode='a', header=False, index=False)

# mostrar sumário
df_game_week_summary

Jornada 16 ainda não foi completa.


StopExecution: 